In [ ]:
#@title ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title データを保存するディレクトリのパスを指定
#@markdown ランタイムに接続しドライブを後、コピーしてくる<br>
#@markdown 後ろの / は追加するので、追記は不要<br>

cwd = "/content/drive/MyDrive/RNAseq/Outputs/\u5B9F\u9A13\u540D" #@param {type:"string"}
cwd = cwd + "/"

# 必要なモジュールをインポートする
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage , dendrogram , fcluster

# カレントディレクトリの変更
import os
os.makedirs(cwd , exist_ok = True)
os.chdir(cwd)
currentdir = os.getcwd()

In [ ]:
#@title 実験名を定義
exp_name = "\u5B9F\u9A13\u540D\u3092\u5165\u529B" #@param {type:"string"}
exp_name = exp_name + "_"


＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
複数のcsvファイルを結合する<br>
pandasのデータフレームとしてcsv2つをそれぞれ読み込み、合わせて別のデータフレームに加算する<br>
読み込んだデータフレームをそれぞれdf1, df2、加算したものをdf3として<br>
df1+df2 =df3とし、df3をcsv出力した後にdf1=df3の処理を行うので、3つ以上のcsvを結合する場合は、df2のpathを書き換えて再度セルを実行すれば良い<br>
csvは毎回上書き出力するので、途中のファイルは残らない

In [ ]:
#@title df1のパスを入力
df1_path = "Kallisto\u306E\u51FA\u529B\u3092\u5F8C\u51E6\u7406\u3057\u305FTPMsum.csv\u30D5\u30A1\u30A4\u30EB\u3092\u5165\u529B(1\u30B5\u30F3\u30D7\u30EB\u76EE)" #@param {type:"string"}
df1 = pd.read_csv(df1_path)
print(df1)

In [ ]:
#@title df2のパスを入力
#@markdown df3を計算してcsv出力、df1の再定義までを行う
df2_path = "Kallisto\u306E\u51FA\u529B\u3092\u5F8C\u51E6\u7406\u3057\u305FTPMsum.csv\u30D5\u30A1\u30A4\u30EB\u3092\u5165\u529B(2\u30B5\u30F3\u30D7\u30EB\u76EE\u4EE5\u964D)" #@param {type:"string"}
df2 = pd.read_csv(df2_path)
df3 = pd.merge(df1 , df2 , on = ["Geneid" , "product"] , how = "outer")
df3 = df3.fillna(0)

df3.to_csv(exp_name + "contact.csv" , index = False)
df1 = df3
print(df3)

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
データを扱いやすいようにGeneidとproductをindex化する

In [ ]:
#@title 2列をindex化、残った行名を表示
df3 = df3.set_index(["Geneid" , "product"])
print("存在している列名は次の通り")
print(list(df3))

クラスタリングとヒートマップでデータを確認する<br>
全データを対象にしたものと、TPMの平均が1以上あるもののみを対象としたものを出力する

In [ ]:
#@title 全データ - クラスタリング図
df_T = df3.T

linkage_result = linkage(df_T , method = "ward" , metric = "euclidean")
plt.figure(num = None , figsize = (16 , 9) , dpi = 200 , facecolor = "w" , edgecolor = "k")
dendrogram(linkage_result , labels = list(df_T.index))
plt.savefig(exp_name + "クラスタリング.png")
plt.show()